In [1]:
import numpy as np
import random
import os

from envs.GridWorld import DrunkenWalkEnv

In [7]:
# 1. Environment setup
env = DrunkenWalkEnv()

Bellman Optimality Equation
$$V(s) = \max_a \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V(s')]$$

In [8]:
# --------------------- Value Iteration ---------------------
def value_iteration(env, theta=1e-5, discount_factor=0.9, max_iter=1000):
    """
    Classic Value Iteration for finite MDPs.
    env.P: dictionary of transition probabilities
           env.P[state][action] = [(prob, next_state, reward, done), ...]
    """
    V = np.zeros(env.nS)
    for i in range(max_iter):
        delta = 0
        for s in range(env.nS):
            # Bellman optimality update
            Q_sa = []
            for a in range(env.nA):
                q = 0
                for prob, next_state, reward, done in env.P[s][a]:
                    q += prob * (reward + discount_factor * V[next_state])
                Q_sa.append(q)
            v_new = max(Q_sa)
            delta = max(delta, abs(v_new - V[s]))
            V[s] = v_new
        if delta < theta:
            print(f"Converged at iteration {i}")
            break

    # Derive policy
    policy = np.zeros(env.nS, dtype=int)
    for s in range(env.nS):
        Q_sa = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[s][a]:
                Q_sa[a] += prob * (reward + discount_factor * V[next_state])
        policy[s] = np.argmax(Q_sa)
    return V, policy

V, policy = value_iteration(env)
print("Optimal state values:\n", V.reshape(-1))
print("Derived policy (action index per state):\n", policy)

Converged at iteration 18
Optimal state values:
 [4.83386696 5.04961066 5.7361052  5.0005887  5.47827117 5.16834241
 6.71053812 4.67266756 6.27788368 7.27505679 8.09006564 5.88396006
 6.07553822 8.3082519  9.59132517 0.        ]
Derived policy (action index per state):
 [1 2 1 0 1 1 1 0 2 1 1 0 2 2 2 0]


In [10]:
def test(env, policy, num_episode=10):
    rewards = []
    success_count = 0

    for episode in range(num_episode):
        state, _ = env.reset()  # unpack tuple
        ep_reward = 0
        done = False

        while not done:
            action = policy[state]  # state is now an int
            next_state, reward, done, trunc, _ = env.step(action)
            ep_reward += reward
            state = next_state

            if done:
                if reward > 0:
                    success_count += 1
                break

        rewards.append(ep_reward)

    print(f"Average reward: {np.mean(rewards):.2f}")
    print(f"Success rate: {success_count / num_episode * 100:.2f}%")

test(env, policy)

Average reward: 5.50
Success rate: 70.00%


In [13]:
import gymnasium as gym
import numpy as np

# --- Environment Setup ---
env = gym.make("FrozenLake-v1", is_slippery=True)
nA = env.action_space.n
nS = env.observation_space.n

# --- Helper Function: Evaluate state-action value ---
def eval_state_action(V, s, a, gamma=0.99):
    return np.sum([p * (r + gamma * V[s_next]) for p, s_next, r, _ in env.P[s][a]])

# --- POLICY ITERATION ---
def policy_evaluation(V, policy, eps=1e-4, gamma=0.99):
    while True:
        delta = 0
        for s in range(nS):
            old_v = V[s]
            V[s] = eval_state_action(V, s, policy[s], gamma)
            delta = max(delta, abs(old_v - V[s]))
        if delta < eps:
            break

def policy_improvement(V, policy, gamma=0.99):
    policy_stable = True
    for s in range(nS):
        old_a = policy[s]
        policy[s] = np.argmax([eval_state_action(V, s, a, gamma) for a in range(nA)])
        if old_a != policy[s]:
            policy_stable = False
    return policy_stable

def policy_iteration(env, gamma=0.99):
    V = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)
    it = 0
    while True:
        policy_evaluation(V, policy, gamma=gamma)
        stable = policy_improvement(V, policy, gamma=gamma)
        it += 1
        if stable:
            break
    print(f"✅ Policy iteration converged in {it} iterations")
    return V, policy

# --- VALUE ITERATION ---
def value_iteration(env, eps=1e-4, gamma=0.99):
    V = np.zeros(nS)
    it = 0
    while True:
        delta = 0
        for s in range(nS):
            old_v = V[s]
            V[s] = max([eval_state_action(V, s, a, gamma) for a in range(nA)])
            delta = max(delta, abs(old_v - V[s]))
        if delta < eps:
            break
        it += 1
    print(f"✅ Value iteration converged in {it} iterations")
    return V

# --- Test policy ---
def run_episodes(env, V=None, policy=None, num_games=100, gamma=0.99):
    tot_rew = 0
    for _ in range(num_games):
        state, _ = env.reset()
        done = False
        while not done:
            if policy is not None:
                action = policy[state]
            else:
                action = np.argmax([eval_state_action(V, state, a, gamma) for a in range(nA)])
            state, reward, done, trunc, _ = env.step(action)
            tot_rew += reward
    print(f"🏆 Won {tot_rew} of {num_games} games! ({tot_rew/num_games*100:.1f}% win rate)")

# --- Run experiments ---
V_pi, policy = policy_iteration(env)
run_episodes(env, policy=policy)

V_vi = value_iteration(env)
run_episodes(env, V=V_vi)


✅ Policy iteration converged in 7 iterations


AttributeError: module 'numpy' has no attribute 'bool8'